# Sinopia testing of `bibframe2marc.xsl`


Sets-up notebook environment and initializes three RDF namespaces and prints the current date.

In [18]:
import datetime
import io
import rdflib
import requests
import pymarc
import sys
from lxml import etree
print(datetime.datetime.utcnow())

BF = rdflib.Namespace("http://id.loc.gov/ontologies/bibframe/")
BFLC = rdflib.Namespace("http://id.loc.gov/ontologies/bflc/")
SINOPIA = rdflib.Namespace("http://sinopia.io/vocabulary/")
XML_NS = {"bf": "http://id.loc.gov/ontologies/bibframe/"}


2019-12-18 21:02:27.510815


In [17]:
# Function binds three of the specific RDF namespaces used by Sinopia.
def init_ns(graph):
    graph.namespace_manager.bind("bf", BF)
    graph.namespace_manager.bind("bflc", BFLC)
    graph.namespace_manager.bind("sinopia", SINOPIA)
    
# Function takes a list Sinopia URIs and returns a RDF Graph
def triples2record(uris):
    record_graph = rdflib.Graph()
    init_ns(record_graph)
    for uri in uris:
        get_request = requests.get(uri)
        raw_utf8 = get_request.text.encode(get_request.encoding).decode()
        record_graph.parse(data=raw_utf8, format='turtle')
    return record_graph

# Function takes a graph, serializes to XML, and the returns the transformed MARC XML
def graph2marcXML(graph):
    rdf_xml = etree.XML(graph.serialize(format='pretty-xml'))
    try:
        return bf2marc_transform(rdf_xml)
    except:
        print(f"Error transforming graph to {sys.exc_info()[0]}")
    
# Function takes a list of Sinopia URLs, creates graphs for each, serializes to XML before, combining
# back into a single XML document for transformation 
def graph2RDFXML(uris):
    for uri in uris:
        graph = triples2record(uri)
        rdf_xml = etree.XML(graph.serialize(format='pretty-xml'))
        yield rdf_xml

# Function takes a MARC XML document and returns the MARC 21 equalivant
def marcXMLto21(marcXML):
    reader = io.StringIO(etree.tostring(marcXML).decode())
    marc_records = pymarc.parse_xml_to_array(reader)
    # Should only be 1 record in the list
    return marc_records[0]
    
# Function takes a Sinopia URI that has a relative URI, replaces relative with absolute, and returns 
# new graph
def update_abs_url(uri):
    get_request = requests.get(uri)
    raw_utf8 = get_request.text.encode(get_request.encoding).decode()
    raw_utf8 = raw_utf8.replace('<>', f"<{uri}>")
    graph = rdflib.Graph()
    init_ns(graph)
    graph.parse(data=raw_utf8, format='turtle')
    return graph

# Function takes a graph with a URI bf:Work with a nested bf:Instance serializes the output to XML and then modifies 
# XML with the expected structure for the bibframe2marc transformation
def nestedInstance(graph):

    rdf_xml = etree.XML(graph.serialize(format='pretty-xml'))
    hasInstance = rdf_xml.find("bf:Work/bf:hasInstance", XML_NS)
    bfInstance = hasInstance.find("bf:Instance", XML_NS)
    # Delete the instance from the hasInstance element
    hasInstance.remove(bfInstance)
    # Creates relationship betweemn bf:hasInstance and the instance
    node_id = bfInstance.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}nodeID"]
    hasInstance.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource"] = node_id
    # Adds rdf:about to bfInstance
    bfInstance.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about"] = node_id
    # Adds the instance back as a top-level element
    rdf_xml.append(bfInstance)
    return rdf_xml

# Function takea a graph with an bf:Instance with one or more nested bf:Works, serializes to XML and then modifies
# the XML to the expected structure for the bibframe2marc xslt
def nestedWork(graph):
    rdf_xml = etree.XML(graph.serialize(format='pretty-xml'))
    instanceOfs = rdf_xml.findall("bf:instanceOf", XML_NS)
    for elem in instanceOfs:
        bfWork = elem.find("bf:Work", XML_NS)
        work_node_id = bfWork.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}nodeID"]
        elem.attrib["{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource"] = work_node_id
        elem.remove(bfWork)
        rdf_xml.append(bfWOrk)
    return rdf_xml

# Opens the XSLT as a file and creates an lxml XSLT object.
with open("../src/xslt/bibframe2marc.xsl", "rb") as fo:
    xml = etree.XML(fo.read())
    bf2marc_transform = etree.XSLT(xml)

# Examples -- Stanford University
We are using a number of Stanford's BIBFRAME Instances and Works in Sinopia that use both nested and unnested resource templates. 

For some of the comparisions, we are using older resources that were created prior to the Sinopia PR#[1868](https://github.com/LD4P/sinopia_editor/pull/1868) that stores relative urls (i.e. `<>`) in the resource's RDF instead of the resource's absolute URI. The `update_abs_url` function does a quick string replacement with the absolute URI and returns a graph.

In [19]:
stanford_nested = update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/d2d48e03-c87f-430d-8708-e8d3d8489105')
print(stanford_nested.serialize(format='pretty-xml').decode())

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:bf="http://id.loc.gov/ontologies/bibframe/"
  xmlns:bflc="http://id.loc.gov/ontologies/bflc/"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:sinopia="http://sinopia.io/vocabulary/"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <bf:Instance rdf:about="https://trellis.stage.sinopia.io/repository/stanford/d2d48e03-c87f-430d-8708-e8d3d8489105">
    <bf:provisionActivityStatement xml:lang="en">Texcoco, Estado de México : Colegio de Postgraduados ; San José, Costa Rica : IICA, 2017</bf:provisionActivityStatement>
    <bf:responsibilityStatement xml:lang="es">coordinación editorial, Víctor M. Villalobos Arámbula, Franklin Marín Vargas, Federico Sancho Guevara, J. Cruz García Albarado</bf:responsibilityStatement>
    <bf:hasSeries>
      <bf:Instance rdf:nodeID="f42d162bc290e4e7a9efe2464401742cfb3">
        <bf:seriesEnumeration xml:lang="en">88</bf:seriesEnumeration>
        <bf:seriesStatement xml:lang="es"

In [20]:
stanford_nested_xml = nestedWork(stanford_nested)
print(etree.tostring(stanford_nested_xml, pretty_print=True).decode())

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Instance rdf:about="https://trellis.stage.sinopia.io/repository/stanford/d2d48e03-c87f-430d-8708-e8d3d8489105">
    <bf:provisionActivityStatement xml:lang="en">Texcoco, Estado de M&#233;xico : Colegio de Postgraduados ; San Jos&#233;, Costa Rica : IICA, 2017</bf:provisionActivityStatement>
    <bf:responsibilityStatement xml:lang="es">coordinaci&#243;n editorial, V&#237;ctor M. Villalobos Ar&#225;mbula, Franklin Mar&#237;n Vargas, Federico Sancho Guevara, J. Cruz Garc&#237;a Albarado</bf:responsibilityStatement>
    <bf:hasSeries>
      <bf:Instance rdf:nodeID="f42d162bc290e4e7a9efe2464401742cfb3">
        <bf:seriesEnumeration xml:lang="en">88</bf:seriesEnumeration>
        <bf:seriesStatement xml:lang="es">Biblioteca b&#

In [21]:
stanford_nested_marc_xml = bf2marc_transform(stanford_nested_xml)
stanford_nested_marc21 = marcXMLto21(stanford_nested_marc_xml)
print(stanford_nested_marc21)

=LDR       nam a22     uu 4500
=001  idp140712519057656
=003  DLC
=008  700101|||||||||xx\\\\\\\\\\\\|||\|\\\\\|
=020  \\$a9786077153511
=040  \\
=245  10$aSemblanzas de la agricultura de las Américas$ccoordinación editorial, Víctor M. Villalobos Arámbula, Franklin Marín Vargas, Federico Sancho Guevara, J. Cruz García Albarado$c
=246  11$aPortrayal of agriculture in the Americas
=250  \\$aPrimera edición
=264  \1$bColegio de Postgraduados (Mexico),$c2017.
=264  \4$c2017.
=300  \\$a203 pages$c31 cm
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191218140848.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



For the unnested testing, we use the same BF Instance (https://trellis.stage.sinopia.io/repository/stanford/e91d50ce-b863-457a-824e-7bd0f9c8a9e7) and tries first BF Work (https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3) and displays the MARC 21 result for comparision. 

The BF Work (https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3) has an incorrect predicate URL for of `http://bibframe.org/ontologies/Contribution` for the contribution class that results in the contribution fields to not transform correctly to the equalivant MARC fields (100, etc.) in the transformation.

In [8]:
stanford_instance = update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/e91d50ce-b863-457a-824e-7bd0f9c8a9e7')
stanford_work1 =  update_abs_url('https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3')
instance_xml = etree.XML(stanford_instance.serialize(format='pretty-xml'))
work1_xml = etree.XML(stanford_work1.serialize(format='pretty-xml'))
print(etree.tostring(work1_xml).decode())

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:ns1="http://bibframe.org/ontologies/" xmlns:sinopia="http://sinopia.io/vocabulary/">
  <bf:Work rdf:about="https://trellis.stage.sinopia.io/repository/stanford/fbe7b462-2894-44c4-bd56-c8dde8c568a3">
    <bf:originDate xml:lang="en">2017</bf:originDate>
    <bf:contribution>
      <ns1:Contribution rdf:nodeID="f4e098454e8354072979bcc9aeb287943b5">
        <ns1:Contribution rdf:resource="http://id.loc.gov/vocabulary/relators/edt"/>
        <ns1:agent>Mar&#237;n Vargas, Franklin </ns1:agent>
      </ns1:Contribution>
    </bf:contribution>
    <bf:subject rdf:resource="http://id.loc.gov/authorities/subjects/sh85002415"/>
    <bf:geographicCoverage>America </bf:geographicCoverage>
    <bf:language rdf:resource="http://id.loc.gov/vocabulary/languages/spa"/>
    <bf:title>
      <bf:Title rdf:nodeID="f4e098454e8354072979bcc9aeb287943b4">
        <bf:mainTitle xml:lang="es"

## Un-nested Work and Instance Examples
Sinopia work URL of 'https://trellis.stage.sinopia.io/repository/stanford/2344e198-4306-45eb-ac55-e8a01836820b' with a corresponding 
instance URL  = 'https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf'


Last week noticed that the RDF XML produced from a combined graph of the BF Work and BF Instance results in the BF Instance `bf:instanceOf` to have as a child element the complete `bf:Work`. However, the `bibframe2marc` transformation expects the `bf:Work` to be at the same level as the `bf:Instance` as evidenced by the the sample XML from the Library of Congress. 

To replicate this XML structure, the BF Work and BF Instanceis serialized as XML first and then creating a combined document with both of these XML elements at the top-level.

In [13]:
work_graph = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/2344e198-4306-45eb-ac55-e8a01836820b'])
instance_graph = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf'])

record_xml = etree.XML(instance_graph.serialize(format='pretty-xml'))

In [14]:
work_xml = etree.XML(work_graph.serialize(format='pretty-xml'))
work_element = work_xml.find("bf:Work", XML_NS)
for child in work_xml.iterchildren():
    record_xml.append(child)
print(etree.tostring(record_xml).decode())

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bflc="http://id.loc.gov/ontologies/bflc/">
  <bf:Instance rdf:about="https://trellis.stage.sinopia.io/repository/stanford/63f532de-65d3-4da4-bae3-9e981bd12eaf">
    <bf:title>
      <bf:Title rdf:nodeID="f84bbf1a5fcb2452690adbe9b11b07b91b4">
        <bf:mainTitle xml:lang="eng">&#1581;&#1602;&#1608;&#1604; &#1593;&#1610;&#1587;&#1609; &#1581;&#1587;&#1606; &#1575;&#1604;&#1610;&#1575;&#1587;&#1585;&#1610;</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:carrier>
      <rdf:Description rdf:about="http://id.loc.gov/vocabulary/carriers/nc">
        <rdfs:label>volume</rdfs:label>
      </rdf:Description>
    </bf:carrier>
    <bf:title>
      <bf:ParallelTitle rdf:nodeID="f84bbf1a5fcb2452690adbe9b11b07b91b6">
        <bf:mainTitle xml:lang="eng">Fields, Issa Hassan a

In [22]:
marc_xml = bf2marc_transform(record_xml)
marc21 = marcXMLto21(marc_xml)

In [23]:
print(marc21)

=LDR       nam a22     uu 4500
=001  idp140712487602056
=003  DLC
=008  700101|||||||||xx\\\\\\\\\\\\|||\|\ara\|
=040  \\
=050  \0$aPJ7874.A78$bZ76 2018
=522  \\
=100  1\$4http://id.loc.gov/vocabulary/relators/aut
=245  10$aحقول عيسى حسن الياسري$cاعداد وتقديم فاطمه خليفه مؤذن
=246  11$aFields, Issa Hassan al-Yasiri
=250  \\$aالطبعه الأولى
=264  \1$bMuʼassasah al-ʻArabīyah lil-Dirāsāt wa-al-Nashr,$c2018.
=300  \\$a304 pages$c24 cm
=653  \\$aArabic poetry--Iraq--History and criticism
=653  \\$aArabic poetry--20th century--History and criticism
=653  \\$aCriticism and interpretation
=653  \\$aYāsirī, ʻĪsá Ḥasan
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191218141206.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



In [47]:
nested_url = "https://trellis.stage.sinopia.io/repository/stanford/e7270251-1654-4a00-8680-9d7474fcc32f"
graph = triples2record([nested_url])
rdfXML = etree.XML(graph.serialize(format='pretty-xml'))

In [37]:
hasInstance = rdfXML.find("bf:Work/bf:hasInstance", {"bf": "http://id.loc.gov/ontologies/bibframe/"})

In [40]:
bfInstance = hasInstance.find("bf:Instance", {"bf": "http://id.loc.gov/ontologies/bibframe/"})

{'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}nodeID': 'f5f0fe331ad944143ad694955b682faa9b1'}


In [50]:
flatten_xml = nestedInstance(graph)

In [51]:
marc_xml = bf2marc_transform(flatten_xml)
marc21 = marcXMLto21(marc_xml)
print(marc21)

=LDR       nam a22     uu 4500
=001  idp140274922897304
=003  DLC
=008  700101|||||||||xx\\\\\\\\\\\\|||\|\ara\|
=040  \\
=050  \0$aPJ7874.A78$bZ76 2018
=100  1\$4http://id.loc.gov/vocabulary/relators/aut
=245  10$aحقول عيسى حسن الياسري$cاعداد وتقديم فاطمة خليفه مؤذن
=246  11$aFields, Issa Hassan al-Yasiri
=250  \\$aالطبعة الأولى
=300  \\$a304 pages$c24 cm
=653  \\$aCriticism and interpretation
=653  \\$aArabic poetry--20th century--History and criticism
=653  \\$aArabic poetry--Iraq--History and criticism
=653  \\$aYāsirī, ʻĪsá Ḥasan
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191216144244.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



In [52]:
print(etree.tostring(flatten_xml, pretty_print=True).decode())

<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
  <bf:Work rdf:about="https://trellis.stage.sinopia.io/repository/stanford/e7270251-1654-4a00-8680-9d7474fcc32f">
    <bf:originPlace>
      <rdf:Description rdf:about="http://id.loc.gov/authorities/names/n83068103">
        <rdfs:label>Beirut (Lebanon)</rdfs:label>
      </rdf:Description>
    </bf:originPlace>
    <bf:hasInstance rdf:resource="f279a41d2c56a4beeb2fa229e5c62a2a1b1"/>
    <bf:subject>
      <rdf:Description rdf:about="http://id.loc.gov/authorities/subjects/sh99005576">
        <rdfs:label>Criticism and interpretation</rdfs:label>
      </rdf:Description>
    </bf:subject>
    <bf:title>
      <bf:Title rdf:nodeID="f279a41d2c56a4beeb2fa229e5c62a2a1b8">
        <bf:mainTitle xml:lang="eng">&#1581;&#1602;&#1608;&#160

### Nested Work Example 2

In [52]:
s_nested2 = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/3897dde4-0393-4600-a0da-cdd7ca0fe521'])
print(s_nested2.serialize(format='turtle').decode())
s_nested2_xml = graph2marcXML(s_nested2)
s_nested2_mrc21 = marcXMLto21(s_nested2_xml)
print(s_nested2_mrc21)

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://trellis.stage.sinopia.io/reposi

In [55]:
work_unnested_3 = triples2record(['https://trellis.stage.sinopia.io/repository/stanford/9508a59e-232b-47ca-901a-2982b3e89013',
                                  'https://trellis.stage.sinopia.io/repository/stanford/a6f2db0c-2162-4a46-9c41-878240ae0d36'])
print(work_unnested_3.serialize(format='turtle').decode())

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://id.loc.gov/authorities/genreForm

# Example -- University of California Davis
Repeat the steps in example one to create a combined BF Work and Instance Graph. Also printing out the number of triples in each graph after it is parsed into the final graph. In the resulting serialization in Turtle you can see the BF Work and Instance is represented as top-level subject edges. The Trellis GET response text was encoded in **ISO-8859-1** and needed to be decoded to **utf-8**.

In [6]:
davis_record = triples2record(["https://trellis.stage.sinopia.io/repository/ucdavis/be99291f-ff37-4dd4-bc71-8ff4ac46d3fb",
                               "https://trellis.stage.sinopia.io/repository/ucdavis/307b37cd-980a-436c-881b-6ed1aa0afeeb"])


print(davis_record.serialize(format='turtle').decode())

@prefix acl: <http://www.w3.org/ns/auth/acl#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix memento: <http://mementoweb.org/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://schema.org/> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://trellis.stage.sinopia.io/reposi

### RDF-XML Serialization and MARC Transformation
With the new RDF graph called `davis_record`, we can then serialize it as RDF-XML and pass it through `bf2marc_transform` XSLT to produce the MARC XML in `davis_marc_xml`.

In [12]:
davis_marc_xml = graph2marcXML(davis_record)
print(davis_marc_xml)

<?xml version="1.0"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim" xmlns:fn="http://www.w3.org/2005/xpath-function">
  <marc:leader>     nam a22     uu 4500</marc:leader>
  <marc:controlfield tag="001">idp140491395026712</marc:controlfield>
  <marc:controlfield tag="003">DLC</marc:controlfield>
  <marc:controlfield tag="008">19121310|||||||||xx            ||| |     |</marc:controlfield>
  <marc:datafield tag="020" ind1=" " ind2=" ">
    <marc:subfield code="a">9787502744878</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="245" xml:lang="chi" ind1="1" ind2="0">
    <marc:subfield code="a">中國古動物</marc:subfield>
    <marc:subfield code="c">關键</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="jpn" ind1="1" ind2="1">
    <marc:subfield code="a">中国古動物</marc:subfield>
  </marc:datafield>
  <marc:datafield tag="246" xml:lang="eng" ind1="1" ind2="1">
    <marc:subfield code="a">The fossil animals of China</marc:subfield>
  </marc:datafield>
  <marc:

Use `pymarc` to create MARC21 format.

In [15]:
marc_record = marcXMLto21(davis_marc_xml)
print(marc_record)

=LDR       nam a22     uu 4500
=001  idp140491395026712
=003  DLC
=008  19121310|||||||||xx\\\\\\\\\\\\|||\|\\\\\|
=020  \\$a9787502744878
=245  10$a中國古動物$c關键
=246  11$a中国古動物
=246  11$aThe fossil animals of China
=250  \\$a第1版
=264  \1$c1998.
=300  \\$a197 pages$c29 cm
=500  \\$aText in Chinese, English, and Japanese ; nomenclature also in Latin
=500  \\$aIncludes index in Latin
=884  \\$aDLC bibframe2marc-xsl v0.1.0-SNAPSHOT$g20191213104714.0$qDLC$uhttps://github.com/lcnetdev/bibframe2marc-xsl



## Example from Library of Congress
Copied the BIBFRAME RDF-XML from the comparsion tool-kit at (http://id.loc.gov/tools/bibframe/comparebf-lccn/2004701771.xml) and directly parsed and transformed the XML to MARC.

In [56]:

raw_lcc_record = """<rdf:RDF xmlns:bf = "http://id.loc.gov/ontologies/bibframe/" xmlns:bflc = "http://id.loc.gov/ontologies/bflc/" xmlns:madsrdf = "http://www.loc.gov/mads/rdf/v1#" xmlns:rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs = "http://www.w3.org/2000/01/rdf-schema#" >
<bf:Work rdf:about="http://bibframe.example.org/13678831#Work" >
<bf:adminMetadata >
<bf:AdminMetadata >
<bf:generationProcess >
<bf:GenerationProcess >
<rdfs:label >DLC marc2bibframe2 v1.5.1: 2019-12-12T11:11:08-05:00</rdfs:label>
</bf:GenerationProcess>
</bf:generationProcess>
<bf:status >
<bf:Status >
<rdfs:label >corrected or revised</rdfs:label>
<bf:code >c</bf:code>
</bf:Status>
</bf:status>
<bflc:encodingLevel >
<bflc:EncodingLevel rdf:about="http://id.loc.gov/vocabulary/menclvl/4" >
<rdfs:label >core</rdfs:label>
</bflc:EncodingLevel>
</bflc:encodingLevel>
<bf:descriptionConventions >
<bf:DescriptionConventions rdf:about="http://id.loc.gov/vocabulary/descriptionConventions/aacr" >
<rdfs:label >aacr</rdfs:label>
</bf:DescriptionConventions>
</bf:descriptionConventions>
<bf:identifiedBy >
<bf:Local >
<rdf:value >13678831</rdf:value>
<bf:assigner >
<bf:Agent rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:assigner>
</bf:Local>
</bf:identifiedBy>
<bf:changeDate rdf:datatype="http://www.w3.org/2001/XMLSchema#dateTime" >2007-08-25T03:55:05</bf:changeDate>
<bf:creationDate rdf:datatype="http://www.w3.org/2001/XMLSchema#date" >2004-06-18</bf:creationDate>
<bf:source >
<bf:Source >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent" />
<rdfs:label >DLC-R</rdfs:label>
</bf:Source>
</bf:source>
<bf:source >
<bf:Source >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Agent" />
<rdfs:label >DLC-R</rdfs:label>
</bf:Source>
</bf:source>
<bf:descriptionModifier >
<bf:Agent rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" >
<rdfs:label >DLC</rdfs:label>
</bf:Agent>
</bf:descriptionModifier>
<bf:descriptionAuthentication >
<bf:DescriptionAuthentication rdf:about="http://id.loc.gov/vocabulary/marcauthen/pcc" >
<rdf:value >pcc</rdf:value>
</bf:DescriptionAuthentication>
</bf:descriptionAuthentication>
</bf:AdminMetadata>
</bf:adminMetadata>
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Text" />
<bf:language >
<bf:Language rdf:about="http://id.loc.gov/vocabulary/languages/jpn" />
</bf:language>
<bf:illustrativeContent >
<bf:Illustration rdf:about="http://id.loc.gov/vocabulary/millus/ill" >
<rdfs:label >illustrations</rdfs:label>
</bf:Illustration>
</bf:illustrativeContent>
<bf:supplementaryContent >
<bf:SupplementaryContent rdf:about="http://id.loc.gov/authorities/genreForms/gf2014026048" >
<rdfs:label >bibliographies</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:supplementaryContent >
<bf:SupplementaryContent rdf:about="http://id.loc.gov/vocabulary/msupplcont/index" >
<rdfs:label >Index present</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:identifiedBy >
<bf:Lccn >
<rdf:value >2004701771</rdf:value>
</bf:Lccn>
</bf:identifiedBy>
<bf:geographicCoverage >
<bf:GeographicCoverage rdf:about="http://id.loc.gov/vocabulary/geographicAreas/a-ja" />
</bf:geographicCoverage>
<bf:classification >
<bf:ClassificationLcc >
<bf:source >
<bf:Source rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:source>
<bf:classificationPortion >DS859</bf:classificationPortion>
<bf:itemPortion >.T3626 2004</bf:itemPortion>
</bf:ClassificationLcc>
</bf:classification>
<bf:classification >
<bf:Classification >
<bf:classificationPortion >210.42</bf:classificationPortion>
<bf:source >
<bf:Source >
<rdfs:label >njb/9</rdfs:label>
</bf:Source>
</bf:source>
</bf:Classification>
</bf:classification>
<bf:contribution >
<bf:Contribution >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bflc/PrimaryContribution" />
<bf:agent >
<bf:Agent rdf:about="http://id.loc.gov/authorities/names/nr2004032488" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Person" />
<bflc:name00MatchKey >Takahashi, Kazuki, 1967-</bflc:name00MatchKey>
<bflc:primaryContributorName00MatchKey >Takahashi, Kazuki, 1967-</bflc:primaryContributorName00MatchKey>
<bflc:name00MarcKey >1001 $6880-01$aTakahashi, Kazuki,$d1967-</bflc:name00MarcKey>
<rdfs:label >Takahashi, Kazuki, 1967-</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:role >
<bf:Role rdf:about="http://id.loc.gov/vocabulary/relators/ctb" />
</bf:role>
</bf:Contribution>
</bf:contribution>
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bf:title >
<bf:Title >
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bflc:titleSortKey >Chūsei shōensei to Kamakura Bakufu</bflc:titleSortKey>
<bf:mainTitle >Chūsei shōensei to Kamakura Bakufu</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:supplementaryContent >
<bf:SupplementaryContent >
<rdfs:label >Includes bibliographical references and indexes.</rdfs:label>
</bf:SupplementaryContent>
</bf:supplementaryContent>
<bf:subject >
<bf:Place rdf:about="http://id.loc.gov/authorities/subjects/sh85069454" >
<rdf:type rdf:resource="http://www.loc.gov/mads/rdf/v1#ComplexSubject" />
<rdfs:label >Japan--History--Kamakura period, 1185-1333.</rdfs:label>
<madsrdf:authoritativeLabel >Japan--History--Kamakura period, 1185-1333.</madsrdf:authoritativeLabel>
<madsrdf:isMemberOfMADSScheme rdf:resource="http://id.loc.gov/authorities/subjects" />
<madsrdf:componentList rdf:parseType="Collection" >
<madsrdf:Geographic rdf:about="http://id.loc.gov/authorities/names/n78089021" >
<madsrdf:authoritativeLabel >Japan</madsrdf:authoritativeLabel>
</madsrdf:Geographic>
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85061212" >
<madsrdf:authoritativeLabel >History</madsrdf:authoritativeLabel>
</madsrdf:Topic>
<madsrdf:Temporal >
<madsrdf:authoritativeLabel >Kamakura period, 1185-1333</madsrdf:authoritativeLabel>
</madsrdf:Temporal>
</madsrdf:componentList>
<bf:source >
<bf:Source >
<bf:code >lcsh</bf:code>
</bf:Source>
</bf:source>
</bf:Place>
</bf:subject>
<bf:subject >
<bf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh2008107312" >
<rdf:type rdf:resource="http://www.loc.gov/mads/rdf/v1#ComplexSubject" />
<rdfs:label >Manors--Japan--History.</rdfs:label>
<madsrdf:authoritativeLabel >Manors--Japan--History.</madsrdf:authoritativeLabel>
<madsrdf:isMemberOfMADSScheme rdf:resource="http://id.loc.gov/authorities/subjects" />
<madsrdf:componentList rdf:parseType="Collection" >
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85080612" >
<madsrdf:authoritativeLabel >Manors</madsrdf:authoritativeLabel>
</madsrdf:Topic>
<madsrdf:Geographic rdf:about="http://id.loc.gov/authorities/names/n78089021" >
<madsrdf:authoritativeLabel >Japan</madsrdf:authoritativeLabel>
</madsrdf:Geographic>
<madsrdf:Topic rdf:about="http://id.loc.gov/authorities/subjects/sh85061212" >
<madsrdf:authoritativeLabel >History</madsrdf:authoritativeLabel>
</madsrdf:Topic>
</madsrdf:componentList>
<bf:source >
<bf:Source >
<bf:code >lcsh</bf:code>
</bf:Source>
</bf:source>
</bf:Topic>
</bf:subject>
<bf:contribution >
<bf:Contribution >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bflc/PrimaryContribution" />
<bf:agent >
<bf:Agent rdf:about="http://id.loc.gov/authorities/names/nr2004032488" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Person" />
<bflc:name00MatchKey >高橋一樹, 1967-</bflc:name00MatchKey>
<bflc:primaryContributorName00MatchKey >高橋一樹, 1967-</bflc:primaryContributorName00MatchKey>
<bflc:name00MarcKey >8801 $6100-01/$1$a高橋一樹,$d1967-</bflc:name00MarcKey>
<rdfs:label xml:lang="ja-jpan" >高橋一樹, 1967-</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:role >
<bf:Role rdf:about="http://id.loc.gov/vocabulary/relators/ctb" />
</bf:role>
</bf:Contribution>
</bf:contribution>
<bf:title >
<bf:Title >
<rdfs:label xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</rdfs:label>
<bflc:titleSortKey >中世荘園制と鎌倉幕府</bflc:titleSortKey>
<bf:mainTitle xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:hasInstance rdf:resource="http://bibframe.example.org/13678831#Instance" />
</bf:Work>
<bf:Instance rdf:about="http://bibframe.example.org/13678831#Instance" >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Print" />
<bf:issuance >
<bf:Issuance rdf:about="http://id.loc.gov/vocabulary/issuance/mono" />
</bf:issuance>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:date rdf:datatype="http://id.loc.gov/datatypes/edtf" >2004</bf:date>
<bf:place >
<bf:Place rdf:about="http://id.loc.gov/vocabulary/countries/ja" />
</bf:place>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:identifiedBy >
<bf:Lccn >
<rdf:value >2004701771</rdf:value>
</bf:Lccn>
</bf:identifiedBy>
<bf:identifiedBy >
<bf:Isbn >
<rdf:value >4827311838</rdf:value>
</bf:Isbn>
</bf:identifiedBy>
<bf:identifiedBy >
<bf:Local >
<rdf:value >DCLP04-B11050</rdf:value>
<bf:source >
<bf:Source >
<rdfs:label >CStRLIN</rdfs:label>
</bf:Source>
</bf:source>
</bf:Local>
</bf:identifiedBy>
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bf:title >
<bf:Title >
<rdfs:label >Chūsei shōensei to Kamakura Bakufu</rdfs:label>
<bflc:titleSortKey >Chūsei shōensei to Kamakura Bakufu</bflc:titleSortKey>
<bf:mainTitle >Chūsei shōensei to Kamakura Bakufu</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:responsibilityStatement >Takahashi Kazuki cho</bf:responsibilityStatement>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:place >
<bf:Place >
<rdfs:label >Tōkyō</rdfs:label>
</bf:Place>
</bf:place>
<bf:agent >
<bf:Agent >
<rdfs:label >Hanawa Shobō</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:date >2004</bf:date>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:provisionActivityStatement >Tōkyō : Hanawa Shobō, 2004.</bf:provisionActivityStatement>
<bf:extent >
<bf:Extent >
<rdfs:label >vii, 433, 31 p.</rdfs:label>
</bf:Extent>
</bf:extent>
<bf:note >
<bf:Note >
<bf:noteType >Physical details</bf:noteType>
<rdfs:label >ill.</rdfs:label>
</bf:Note>
</bf:note>
<bf:dimensions >22 cm.</bf:dimensions>
<bf:title >
<bf:Title >
<rdfs:label xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</rdfs:label>
<bflc:titleSortKey >中世荘園制と鎌倉幕府</bflc:titleSortKey>
<bf:mainTitle xml:lang="ja-jpan" >中世荘園制と鎌倉幕府</bf:mainTitle>
</bf:Title>
</bf:title>
<bf:responsibilityStatement xml:lang="ja-jpan" >高橋一樹著</bf:responsibilityStatement>
<bf:provisionActivity >
<bf:ProvisionActivity >
<rdf:type rdf:resource="http://id.loc.gov/ontologies/bibframe/Publication" />
<bf:place >
<bf:Place >
<rdfs:label xml:lang="ja-jpan" >東京</rdfs:label>
</bf:Place>
</bf:place>
<bf:agent >
<bf:Agent >
<rdfs:label xml:lang="ja-jpan" >塙書房</rdfs:label>
</bf:Agent>
</bf:agent>
<bf:date xml:lang="ja-jpan" >2004</bf:date>
</bf:ProvisionActivity>
</bf:provisionActivity>
<bf:provisionActivityStatement xml:lang="ja-jpan" >東京 : 塙書房, 2004.</bf:provisionActivityStatement>
<bf:instanceOf rdf:resource="http://bibframe.example.org/13678831#Work" />
<bf:hasItem >
<bf:Item rdf:about="http://bibframe.example.org/13678831#Item050-14" >
<bf:shelfMark >
<bf:ShelfMarkLcc >
<rdfs:label >DS859.T3626 2004</rdfs:label>
<bf:source >
<bf:Source rdf:about="http://id.loc.gov/vocabulary/organizations/dlc" />
</bf:source>
</bf:ShelfMarkLcc>
</bf:shelfMark>
<bf:itemOf rdf:resource="http://bibframe.example.org/13678831#Instance" />
</bf:Item>
</bf:hasItem>
</bf:Instance>
</rdf:RDF>"""

In [105]:
loc_graph = rdflib.Graph()
init_ns(loc_graph)
loc_graph.parse(data=raw_lcc_record, format='xml')
print(loc_graph.serialize(format='turtle').decode())

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix madsrdf: <http://www.loc.gov/mads/rdf/v1#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://bibframe.example.org/13678831#Item050-14> a bf:Item ;
    bf:itemOf <http://bibframe.example.org/13678831#Instance> ;
    bf:shelfMark [ a bf:ShelfMarkLcc ;
            rdfs:label "DS859.T3626 2004" ;
            bf:source <http://id.loc.gov/vocabulary/organizations/dlc> ] .

<http://bibframe.example.org/13678831#Work> a bf:Text,
        bf:Work ;
    rdfs:label "Chūsei shōensei to Kamakura Bakufu" ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel <http://id.loc.gov/vocabulary/menclvl/4> ;
            bf:changeDate "2007-08-

In [63]:
loc_xml = etree.XML(raw_lcc_record)
loc_marc = bf2marc_transform(loc_xml)

In [59]:
loc_marc21 = marcXMLto21(loc_marc)
print(loc_marc21)

=LDR       cam a22     4a 4500
=001  13678831
=003  DLC
=005  20070825035505.0
=008  040618s2004\\\\ja\\\\\\\\\\\\|||\|\jpn\|
=010  \\$a2004701771
=020  \\$a4827311838
=035  \\$a(CStRLIN)DCLP04-B11050
=040  \\$aDLC-R$cDLC-R$dDLC$eaacr
=042  \\$apcc
=043  \\$aa-ja---
=050  \0$aDS859$b.T3626 2004
=084  \\$a210.42$2njb/9
=100  1\$aTakahashi, Kazuki, 1967-$0http://id.loc.gov/authorities/names/nr2004032488$4http://id.loc.gov/vocabulary/relators/ctb
=240  10$aChūsei shōensei to Kamakura Bakufu
=241  10$aChūsei shōensei to Kamakura Bakufu
=245  10$a中世荘園制と鎌倉幕府$c高橋一樹著
=264  \1$ahttp://id.loc.gov/vocabulary/countries/ja$c2004.
=264  \1$aTōkyō:$bHanawa Shobō,$c2004.
=264  \1$a東京:$b塙書房,$c2004.
=300  \\$avii, 433, 31 p.$bill.$c22 cm.
=500  \\$aill.
=525  \\$abibliographies
=525  \\$aIndex present
=525  \\$aIncludes bibliographical references and indexes.
=651  \0$0http://id.loc.gov/authorities/subjects/sh85069454$aJapan$xHistory$0http://id.loc.gov/authorities/subjects/sh85061212$yKamakura period, 1